In [ ]:
import time
import board
import busio
import numpy as np
import cv2
import datetime
from opcua import Client
from adafruit_mlx90640 import MLX90640
from telegram import Bot
import RPi.GPIO as GPIO

In [ ]:
# --- Telegram Configuration
TELEGRAM_BOT_TOKEN = 'YOUR_BOT_TOKEN'  # Replace with your actual bot token
CHAT_ID = 'YOUR_CHAT_ID'              # Replace with the chat ID of the recipient
bot = Bot(token=TELEGRAM_BOT_TOKEN)

In [ ]:
# --- OPC UA Configuration (PLC)
PLC_URL = "opc.tcp://localhost:4840"
NODE_ID = "ns=2;s=OPC_Daten.Anlage_ausschalten"  # Node ID to trigger system shutdown

In [ ]:
# --- MLX90640 Thermal Camera Setup
i2c = busio.I2C(board.SCL, board.SDA, frequency=400000)
mlx = MLX90640(i2c)
mlx.refresh_rate = MLX90640.RefreshRate.4_HZ  # Set to 4Hz frame capture rate

In [ ]:
# --- Threshold and Image Settings
TEMPERATURE_THRESHOLD = 30.0  # Temperature in Celsius
IMAGE_SHAPE = (24, 32)        # MLX90640 resolution

In [ ]:
# --- Buzzer Configuration (Optional)
BUZZER_PIN = 18
GPIO.setmode(GPIO.BCM)
GPIO.setup(BUZZER_PIN, GPIO.OUT)

In [ ]:
def capture_frame():
    """
    Captures a single frame from the thermal camera and returns it as a 2D NumPy array.
    """
    frame = [0] * 768
    mlx.getFrame(frame)
    thermal_array = np.reshape(frame, IMAGE_SHAPE)
    return thermal_array

def save_thermal_image(array, filename):
    """
    Normalizes and resizes the thermal image, then saves it as a .jpg file.
    """
    normalized = cv2.normalize(array, None, 0, 255, cv2.NORM_MINMAX)
    image = np.uint8(normalized)
    resized_image = cv2.resize(image, (320, 240), interpolation=cv2.INTER_LINEAR)
    cv2.imwrite(filename, resized_image)

def send_telegram_alert(temperature, image_path):
    """
    Sends a temperature warning message and image to a Telegram chat.
    """
    alert_message = f" Fire Alert: High temperature detected!\n🌡️ Temperature: {temperature:.2f} °C"
    bot.send_message(chat_id=CHAT_ID, text=alert_message)
    with open(image_path, 'rb') as photo:
        bot.send_photo(chat_id=CHAT_ID, photo=photo)

def shutdown_machine():
    """
    Connects to a PLC using OPC UA and sets the shutdown signal node to True.
    """
    try:
        client = Client(PLC_URL)
        client.connect()
        node = client.get_node(NODE_ID)
        node.set_value(True)
        print("Shutdown signal sent to PLC.")
        client.disconnect()
    except Exception as e:
        print("Error connecting to PLC:", e)

def trigger_buzzer(duration=5):
    """
    Activates the buzzer for a specified number of seconds.
    """
    GPIO.output(BUZZER_PIN, GPIO.HIGH)
    time.sleep(duration)
    GPIO.output(BUZZER_PIN, GPIO.LOW)

def monitor_temperature():
    """
    Main monitoring loop. Continuously reads the thermal camera and responds to high temperatures.
    """
    while True:
        try:
            thermal_data = capture_frame()
            max_temp = np.max(thermal_data)
            print("Max Temperature:", max_temp)

            if max_temp >= TEMPERATURE_THRESHOLD:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                image_filename = f"thermal_alert_{timestamp}.jpg"
                
                save_thermal_image(thermal_data, image_filename)
                send_telegram_alert(max_temp, image_filename)
                trigger_buzzer()
                shutdown_machine()

                time.sleep(30)  # Delay to avoid repeated alerts
            else:
                time.sleep(2)

        except Exception as error:
            print("Error during monitoring:", error)
            time.sleep(5)

# --- Start Monitoring
try:
    monitor_temperature()
except KeyboardInterrupt:
    print("Monitoring stopped by user.")
    GPIO.cleanup()